In [1]:
import timeit

In [20]:
setup_code = """
import torch
import importlib
import grow_muscle
importlib.reload(grow_muscle)
rads_batch = torch.randn(10000,5)
rad_deltas_batch = torch.randn(10000,5)
capital_batch = torch.randn(10000)
efficiency_batch = torch.randn(10000)
"""
num = 1000

test_code1 = """
grow_muscle.grow_old(rads_batch, rad_deltas_batch, capital_batch, efficiency_batch)
"""

times1 = timeit.repeat(setup=setup_code, stmt=test_code1, repeat=3, number=num)

test_code2 = """
grow_muscle.grow_muscle(rads_batch, rad_deltas_batch, capital_batch, efficiency_batch)
"""

times2 = timeit.repeat(setup=setup_code, stmt=test_code2, repeat=3, number=num)

# test_code3 = """
# for i in range(len(rads_batch)):
#     grow_muscle.grow_dumb(rads_batch[i].numpy(), rad_deltas_batch[i].numpy(), capital_batch[i].numpy(), efficiency_batch[i].numpy())
# """

# times3 = timeit.repeat(setup=setup_code, stmt=test_code3, repeat=3, number=num)

print(f'grow_old time: {sum(times1) / len(times1*num)}')
print(f'grow_muscle time: {sum(times2) / len(times2*num)}')
# print(f'grow_dumb time: {sum(times3) / len(times3)}')



grow_old time: 0.0013811915693319559
grow_muscle time: 0.0005479471806659906


In [19]:
times1

[1.4439228330011247, 1.3595647920010379, 1.3719559580058558]

In [4]:
import timeit
setup_code = """
import torch
t = torch.randn(10000)
t2 = torch.randn(10000)
"""

test_code1 = """
t -= t2
"""

test_code2 = """
t.sub_(t2)
"""

times1 = timeit.repeat(setup=setup_code, stmt=test_code1, repeat=3, number=10000)
times2 = timeit.repeat(setup=setup_code, stmt=test_code2, repeat=3, number=10000)

print(f"Average time for 't -= t2': {sum(times1) / len(times1)}")
print(f"Average time for 't.sub_(t2)': {sum(times2) / len(times2)}")


Average time for 't -= t2': 0.014941791669116355
Average time for 't.sub_(t2)': 0.011395708002964966


In [2]:
setup_code = """
import torch
a = torch.randn(10000)
b = torch.randn(10000)
c = torch.randn(10000)
"""

test_code1 = """
a.addcmul_(-b, c)
"""

test_code2 = """
a.sub_(b * c)
"""

test_code3 = """
a -= b * c
"""

times1 = timeit.repeat(setup=setup_code, stmt=test_code1, repeat=3, number=10000)
times2 = timeit.repeat(setup=setup_code, stmt=test_code2, repeat=3, number=10000)
times3 = timeit.repeat(setup=setup_code, stmt=test_code3, repeat=3, number=10000)

print(f"Average time for 'a.addcmul_(-b, c)': {sum(times1) / len(times1)}")
print(f"Average time for 'a.sub_(b * c)': {sum(times2) / len(times2)}")
print(f"Average time for 'a -= b * c': {sum(times3) / len(times3)}")

Average time for 'a.addcmul_(-b, c)': 0.034066513670647204
Average time for 'a.sub_(b * c)': 0.032938444666797295
Average time for 'a -= b * c': 0.033240666661489136


In [3]:
setup_code_unsqueeze = """
import torch
t = torch.randn(10000)
"""

test_code_unsqueeze = """
t.unsqueeze(1)
"""

test_code_none = """
t[:, None]
"""

times_unsqueeze = timeit.repeat(setup=setup_code_unsqueeze, stmt=test_code_unsqueeze, repeat=3, number=10000)
times_none = timeit.repeat(setup=setup_code_unsqueeze, stmt=test_code_none, repeat=3, number=10000)

print(f"Average time for 't.unsqueeze(1)': {sum(times_unsqueeze) / len(times_unsqueeze)}")
print(f"Average time for 't[:, None]': {sum(times_none) / len(times_none)}")


Average time for 't.unsqueeze(1)': 0.005111194666824304
Average time for 't[:, None]': 0.009533514001911195


In [34]:
setup_code_unsqueeze = """
import torch
t = torch.randn(10000)
"""

test_code_where = """
t1 = torch.where(t>0.75, t*t, t)
"""

test_code_control = """
t1 = t.clone()
"""

test_code_all = """
t1 = t.clone()
t1 *= t1
t = torch.where(t>0.75, t1, t)
"""

test_code_batch = """
t1 = t.where(t>0.75, t,)
"""

times_where = timeit.repeat(setup=setup_code_unsqueeze, stmt=test_code_where, repeat=3, number=10000)
times_control = timeit.repeat(setup=setup_code_unsqueeze, stmt=test_code_control, repeat=3, number=10000)
times_all = timeit.repeat(setup=setup_code_unsqueeze, stmt=test_code_all, repeat=3, number=10000)

print(f"Average time for 'torch.where': {sum(times_where) / len(times_where)}")
print(f"Average time for 'control': {sum(times_control) / len(times_control)}")
print(f"Average time for 'all': {sum(times_all) / len(times_all)}")


Average time for 'torch.where': 0.13378423633791195
Average time for 'control': 0.010918666666839272
Average time for 'all': 0.13261576367464536


In [26]:
import torch
t = torch.tensor([1,2,3,4,5])
t1 = t * 1.0

t1.mul_(10)
print(t)

tensor([1, 2, 3, 4, 5])


In [63]:
setup_code = """
import torch
t = torch.randn((10,100,100))
threshold = 2
"""

whole = """
t1 = torch.where(t>2, t, torch.tensor(0.0))
t1*=t1
t1+=t1
t = torch.where(t>2, t1, t)
"""

batch = """
indices = t > threshold
vals = t[indices]
vals *= vals
vals += vals
t[indices] = vals
"""

inplace = """ 
indices = t > threshold
t[indices] *= t[indices]
t[indices] += t[indices]
"""

times_whole = timeit.repeat(setup=setup_code, stmt=whole, repeat=3, number=100)
times_batch = timeit.repeat(setup=setup_code, stmt=batch, repeat=3, number=100)
times_inplace = timeit.repeat(setup=setup_code, stmt=inplace, repeat=3, number=100)

print(f"Average time for 'whole': {sum(times_whole) / len(times_whole)}")
print(f"Average time for 'batch': {sum(times_batch) / len(times_batch)}")
print(f"Average time for 'inplace': {sum(times_inplace) / len(times_inplace)}")
times_whole


Average time for 'whole': 0.029083986339780193
Average time for 'batch': 0.03773087499818454
Average time for 'inplace': 0.10167401400394738


[0.03381441699457355, 0.02547270801733248, 0.027964834007434547]